In [1]:
# System modules
from Queue import Queue
from threading import Thread
import time

# Local modules
import feedparser

# Set up some global variables
num_fetch_threads = 2
job_queue = Queue()

# A real app wouldn't use hard-coded data...
jobs = [i for i in range(100)]


def jobWorker(i, q):
    while True:
        print '%s: Looking for the next enclosure' % i
        job = q.get()
        print '%s: job:' % i, job
        q.task_done()


# Set up some threads to fetch the enclosures
for i in range(num_fetch_threads):
    worker = Thread(target=jobWorker, args=(i, job_queue,))
    worker.setDaemon(True)
    worker.start()

# Download the feed(s) and put the enclosure URLs into
# the queue.
for job in jobs:
    job_queue.put(job)
        
# Now wait for the queue to be empty, indicating that we have
# processed all of the downloads.
print '*** Main thread waiting'
job_queue.join()
print '*** Done'


0: Looking for the next enclosure
1: Looking for the next enclosure
1: job: 0: job:0 
11: Looking for the next enclosure

0: Looking for the next enclosure1: job:
 0: job:2 
31: Looking for the next enclosure

0: Looking for the next enclosure
1: job: 4
 0: job: 5
*** Main thread waiting
1: Looking for the next enclosure
0: Looking for the next enclosure
1: job: 6
0: job: 7
1: Looking for the next enclosure
0: Looking for the next enclosure
1: job: 8
0: job: 9
1: Looking for the next enclosure
0: Looking for the next enclosure
1: job:0: job:  1011

1: Looking for the next enclosure0: Looking for the next enclosure

1: job: 12
0: job: 13
1: Looking for the next enclosure
0: Looking for the next enclosure
1: job: 14
0: job: 15
1: Looking for the next enclosure
0: Looking for the next enclosure
1: job: 16
0: job: 17
1: Looking for the next enclosure
0: Looking for the next enclosure
1: job: 18
0: job:1: Looking for the next enclosure 
191: job:
 0: Looking for the next enclosure20

0: job

In [ ]:
"""Simple example of doing arbitrarily long calculations nicely in Twisted.

This is also a simple demonstration of twisted.protocols.basic.LineReceiver.
"""

from twisted.protocols import basic
from twisted.internet import reactor
from twisted.internet.protocol import ServerFactory

class LongMultiplicationProtocol(basic.LineReceiver):
    """A protocol for doing long multiplications.

    It receives a list of numbers (separated by whitespace) on a line, and
    writes back the answer.  The answer is calculated in chunks, so no one
    calculation should block for long enough to matter.
    """
    def connectionMade(self):
        self.workQueue = []
        
    def lineReceived(self, line):
        try:
            numbers = map(long, line.split())
        except ValueError:
            self.sendLine('Error.')
            return

        if len(numbers) <= 1:
            self.sendLine('Error.')
            return

        self.workQueue.append(numbers)
        reactor.callLater(0, self.calcChunk)

    def calcChunk(self):
        # Make sure there's some work left; when multiple lines are received
        # while processing is going on, multiple calls to reactor.callLater()
        # can happen between calls to calcChunk().
        if self.workQueue:
            # Get the first bit of work off the queue
            work = self.workQueue[0]
    
            # Do a chunk of work: [a, b, c, ...] -> [a*b, c, ...]
            work[:2] = [work[0] * work[1]]
    
            # If this piece of work now has only one element, send it.
            if len(work) == 1:
                self.sendLine(str(work[0]))
                del self.workQueue[0]
            
            # Schedule this function to do more work, if there's still work
            # to be done.
            if self.workQueue:
                reactor.callLater(0, self.calcChunk)


class LongMultiplicationFactory(ServerFactory):
    protocol = LongMultiplicationProtocol


if __name__ == '__main__':
    from twisted.python import log
    import sys
    log.startLogging(sys.stdout)
    reactor.listenTCP(1234, LongMultiplicationFactory())
    reactor.run()


2016-05-09 06:35:01+0530 [-] Log opened.
2016-05-09 06:35:01+0530 [-] LongMultiplicationFactory starting on 1234
2016-05-09 06:35:01+0530 [-] Starting factory <__main__.LongMultiplicationFactory instance at 0x7f25a47ed758>


In [ ]:
from twisted.internet import reactor

def aSillyBlockingMethod(x):
    import time
    time.sleep(2)
    print(x)

reactor.callInThread(aSillyBlockingMethod, "2 seconds have passed")
reactor.run()


2 seconds have passed
